#NLP2

In [1]:
# Importation des bibliothèues nécessaires
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# 1. Charger les données
data = pd.read_csv('/content/tweets_suspect.csv')

In [3]:
data

,message,label
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",1
1,is upset that he can't update his Facebook by ...,1
2,@Kenichan I dived many times for the ball. Man...,1
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",1
...,...,...
59995,"Really wants to go and see 17 again, because Z...",1
59996,@krissa22 Thank you!,1
59997,dreaming of you,1
59998,@TheEllenShow I saw a clip online! good show!,1


In [4]:
data["label"].value_counts()

,count
label,
1,53855
0,6145


In [5]:
df=data

#Impression des tweet qui commence par @, dieze et des sites

In [6]:
# Fonction pour extraire les mots qui commencent par "#"
def extract_mentions(phrase):
    return re.findall(r'#\w+', phrase)

df['mentions_dieze'] = df['message'].apply(extract_mentions)
all_mentions = df['mentions_dieze'].sum()
print(all_mentions)

['#itm', '#therapyfail', '#fb', '#TTSC', '#24', '#gayforpeavy', '#FML', '#3', '#camerafail', '#fb', '#travian', '#wow', '#art', '#fail', '#KrispyKreme', '#fb', '#24', '#the', '#twitter', '#fail', '#CNN', '#1', '#Kulula', '#surface', '#c25k', '#drupal', '#epicfail', '#rails', '#mhbigcatch', '#stuckrecord', '#twitteriffic', '#startrek', '#earthquake', '#php', '#poken', '#error', '#tracy', '#fb', '#dwsr', '#Italy', '#Earthquake', '#fantasysurfer', '#titsuptuesday', '#N85', '#heyxboxlive', '#House', '#babypunch', '#cob', '#ticket', '#fb', '#fb', '#terremoto', '#earthquake', '#italy', '#fail', '#itm', '#AutomationAtaCost', '#gsoc', '#rockbox', '#niceweather', '#Berlin', '#goodlife', '#Ambients', '#theinbetweeners', '#Twilight', '#Summers', '#mmuk09', '#i36', '#spotify', '#42', '#fb', '#pman', '#spbpuk', '#dreams', '#dreams', '#dwsr', '#fb', '#wolframalpha', '#search', '#semantic', '#Virgin', '#liquor', '#Easter', '#NAME', '#liquor', '#Easter', '#WCH', '#tired', '#barcodes', '#nickdarcy', '#

In [ ]:
# Fonction pour extraire les mots qui commencent par "@"
def extract_mentions(phrase):
    return re.findall(r'@\w+', phrase)

df['mentions'] = df['message'].apply(extract_mentions)
all_mentions = df['mentions'].sum()
print(all_mentions)

In [ ]:

# Fonction pour extraire les URLs
def extract_urls(phrase):
    # Expression régulière pour capturer les URLs
    url_pattern = r'(https?://[^\s]+|www\.[^\s]+)'
    return re.findall(url_pattern, phrase)

df['urls'] = df['message'].apply(extract_urls)
all_urls = df['urls'].sum()

print(all_urls)


In [ ]:
df

#Supression des mots commencant par @ , dieze  et les sites

In [ ]:
import re

# Fonction pour supprimer les mots qui commencent par "@" et les sites web
def remove_mentions_urls_hashtags(phrase):
    # Expression régulière pour les mots commençant par "@"
    mention_pattern = r'@\w+'
    # Expression régulière pour les mots commençant par "#"
    hashtag_pattern = r'#\w+'
    # Expression régulière pour les URLs
    url_pattern = r'(https?://[^\s]+|www\.[^\s]+)'
    clean_phrase = re.sub(f'{mention_pattern}|{hashtag_pattern}|{url_pattern}', '', phrase)
    return ' '.join(clean_phrase.split())


df['clean_phrases'] = df['message'].apply(remove_mentions_urls_hashtags)

In [ ]:
df

In [ ]:
data=df

#Suprimmer les ponctuation

In [ ]:
import string
string.punctuation

In [ ]:
def Remove_Punctuation(text):
    result = "".join([ch for ch in text if ch not in string.punctuation])
    return result

data['Content_clean'] = data['clean_phrases'].apply(lambda x: Remove_Punctuation(x))

data.head()

#Tokenization

In [ ]:
import re
def tokenizer(text):
    words = re.split('\W+', text)
    return words

data['Content_tokenized'] = data['Content_clean'].apply(lambda x: tokenizer(x.lower()))

data.head()

#Supprimer les stopword

In [ ]:
import nltk
from nltk.corpus import stopwords

# Télécharger les stopwords
nltk.download('stopwords')

# Charger les stopwords en anglais
en_stopwords = set(stopwords.words('english'))
print(en_stopwords)

In [ ]:
def Remove_Stopwords(content_list):
    result = [word for word in content_list if word not in en_stopwords]
    return result

data['Content_NO_Stopwords'] = data['Content_tokenized'].apply(lambda x: Remove_Stopwords(x))

data.head()

#Lemmatisation

In [ ]:
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()


In [ ]:
def lemmatizing(words):
    result = [wn.lemmatize(word) for word in words]
    return result

data['Content_lemmatized'] = data['Content_NO_Stopwords'].apply(lambda x: lemmatizing(x))

data.head(10)

#Classification

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [ ]:
# 3. Split train-test
X_train, X_test, y_train, y_test = train_test_split(data['Content_lemmatized'], data['label'], test_size=0.2, random_state=42)
X_train = [' '.join(tokens) for tokens in X_train]
X_test = [' '.join(tokens) for tokens in X_test]

In [ ]:
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(X_train)
X_test_counts=count_vect.transform(X_test)
X_train_counts.shape


In [ ]:
X_train

In [ ]:
tfidf_tranformer = TfidfTransformer()
X_train_tfidf = tfidf_tranformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_tranformer.transform(X_test_counts)

In [ ]:
clf=MultinomialNB().fit(X_train_tfidf,y_train)

In [ ]:
y_pred=clf.predict(X_test_tfidf)

In [ ]:
print(metrics.classification_report(y_test,y_pred))#}, target_names=target_names))
print(metrics.confusion_matrix(y_test,y_pred))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(tol=None, n_jobs=-1))]).fit(X_train, y_train)

In [ ]:
y_pred=text_clf.predict(X_test)
print(metrics.classification_report(y_test,y_pred))#}, target_names=target_names))
print(metrics.confusion_matrix(y_test,y_pred))